# 02 - Làm sạch & chuẩn hóa dữ liệu
Notebook này mô tả chi tiết từng bước làm sạch MovieLens: xử lý thiếu/chuẩn hóa, ghép thống kê và xuất file `master.parquet`. Tất cả chú thích được viết bằng tiếng Việt để thuận tiện báo cáo.

In [2]:
from pathlib import Path
import sys
import pandas as pd

PROJECT_ROOT = Path('..').resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
from src import data_ingestion, data_cleaning

RAW_DIR = PROJECT_ROOT / 'data' / 'raw'
PROCESSED_DIR = PROJECT_ROOT / 'data' / 'processed'
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
RAW_DIR

WindowsPath('D:/New folder/data/raw')

## Cấu hình đường dẫn & thư viện
Xác định thư mục dự án, thư mục raw/processed và đảm bảo các thư mục tồn tại trước khi ghi file kết quả.

In [3]:
raw_data = data_ingestion.load_raw_data(RAW_DIR)
master_df = data_cleaning.build_master_frame(raw_data)
master_df.head()

,userId,movieId,rating,timestamp,title,genres,tag_text,year,avg_rating,rating_count,rating_std,genre_list,clean_title
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,fun pixar,1995.0,3.920930,114.54,0.834859,"[Adventure, Animation, Children, Comedy, Fantasy]",Toy Story
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance,moldy old,1995.0,3.259615,52.00,1.054823,"[Comedy, Romance]",Grumpier Old Men
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller,,1995.0,3.946078,102.00,0.817224,"[Action, Crime, Thriller]",Heat
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,mystery serial killer twist ending,1995.0,3.975369,114.54,0.922429,"[Mystery, Thriller]",Seven (a.k.a. Se7en)
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,heist mindfuck suspense thriller tricky twist ...,1995.0,4.237745,114.54,0.800921,"[Crime, Mystery, Thriller]","Usual Suspects, The"


## Thực hiện chuỗi làm sạch
Gọi `build_master_frame` để hợp nhất ratings/movies/tags, xử lý thiếu, loại duplicate và bổ sung thống kê từng phim.

In [4]:
summary = {
    'rows': len(master_df),
    'unique_users': master_df['userId'].nunique(),
    'unique_movies': master_df['movieId'].nunique(),
    'missing_after_clean': master_df.isna().sum().sum(),
}
summary

{'rows': 100836,
 'unique_users': 610,
 'unique_movies': 9724,
 'missing_after_clean': 18}

### Kiểm tra nhanh chất lượng dữ liệu sau làm sạch
Ghi nhận số lượng bản ghi, số user/movie duy nhất và tổng số ô thiếu (kỳ vọng ~0).

In [7]:
master_path = PROCESSED_DIR / 'master.parquet'
master_df.to_parquet(master_path, index=False)
master_path

WindowsPath('D:/New folder/data/processed/master.parquet')

## Lưu master dataset
Xuất toàn bộ khung dữ liệu đã làm sạch dưới dạng `parquet` để các notebook kế tiếp tải vào nhanh hơn.